In [7]:
import pandas as pd
data_train = pd.read_csv('./test.txt')
#data_train.columns = ["tanggal", "tweet"]
data_train.head()

,im feeling rather rotten so im not very ambitious right now;sadness
0,im updating my blog because i feel shitty;sadness
1,i never make her separate from me because i do...
2,i left with my bouquet of red and yellow tulip...
3,i was feeling a little vain when i did this on...
4,i cant walk into a shop anywhere where i do no...


In [8]:
import pandas as pd
import re
from collections import Counter


def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('spellcheck.txt').read()))
# WORDS = Counter(words(open('..\\resources\\spellcheck.txt').read()))

def P(word, N=sum(WORDS.values())): 
    # "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    # "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    # "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    # "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    # "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)] # [('', 'kemarin'), ('k', 'emarin'), ('ke', 'marin'), dst]
    deletes    = [L + R[1:]               for L, R in splits if R] # ['emarin', 'kmarin', 'kearin', dst]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1] # ['ekmarin', 'kmearin', 'keamrin', dst]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters] # ['aemarin', 'bemarin', 'cemarin', dst]
    inserts    = [L + c + R               for L, R in splits for c in letters] # ['akemarin', 'bkemarin', 'ckemarin', dst]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    # "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
  
# print('yg = ', end='')
# coba = correction("yg")
# print(coba)

# editsas = len(set(edits2("bagamana")))
# print(editsas

In [11]:
import re
import modulenorm.modSpellChecker as sc
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

character = ['.',',',';',':','-,','...','?','!','(',')','[',']','{','}','<','>','"','/','\'','#','-','@']
emoticon = [':)',':]','=)',':-)',':(',':[','=(',':-(',':p',':P','=P',':-p',':-P',':D','=D',':-D',':o',':O',':-o',':-O',';)',';-)','8-)','B-)','^_^','-_-','>:o','>:O',':v',':3','8|','B|','8-|','B-|','>:(',':/',':\\',':-/',':-\\',':\'(','O:)',':*',':-*','<3','(y)','(Y)']
remove_charac = ['—','…']

class normalize():
	def enterNormalize(self, text):
		norm_enter = text.replace("\n", " ")
		return norm_enter

	def lowerNormalize(self, text):
		norm_lower = text.lower()
		return norm_lower

	def repeatcharNormalize(self, text):
		for i in range(len(character)):
			charac_long = 5
			while charac_long>=2:
				char = character[i]*charac_long 
				text = text.replace(char,character[i])
				charac_long -= 1
		return text

	def spacecharNormalize(self, text):
		text = re.sub(r'([' + ''.join(map(re.escape, character)) + r'])(?=\S)', r'\1 ', text)
		text = re.sub(r'(\S)([' + ''.join(map(re.escape, character)) + r'])', r'\1 \2', text)
		return text

	def linkNormalize(self, text):
		text = re.sub(r"\s—\s", "", text)
		text = re.sub(r"http\S+", "", text)
		return text

	def wordcNormalize(self, text, loop=2):
		for a in range(loop):
			checkw = False
			for i in range(len(text)):
				if text[i] == '-':
					if text[i-1] == text[i+1]:
						katalengkap = text[i-1]+text[i]+text[i+1]
						pb = i-1
						checkw = True
			if checkw:
				del text[pb]
				del text[pb]
				del text[pb]
				text.insert(pb, katalengkap)
		return text

	def emoticonNormalize(self, text):
		def tighten_emoticon(matchobj):
		    return matchobj.group(0).replace(" ", "")

		REGEX = '|'.join([re.escape(' '.join(x)) for x in emoticon])
		tightened = re.sub(REGEX, tighten_emoticon, text)
		return tightened

	def ellipsisNormalize(self, text):
		text = text.replace('…',' …')
		text = text.replace(' …','')
		return text

	def spellNormalize(self, text):
		spellCheck = []
		for i in text:
			if i not in character:
				j = sc.correction(i)
				spellCheck.append(j)
			else:
				spellCheck.append(i)
		return spellCheck

	def stemmingNormalize(self, text, datatype='sentence'):
		factory = StemmerFactory()
		stemmer = factory.create_stemmer()

		if datatype == 'sentence':
			output = stemmer.stem(text)
			return output
		elif datatype == 'word':
			output = []
			for i in text:
				if i in character or i in emoticon:
					output.append(i)
				else:
					stemmed = stemmer.stem(i)
					output.append(stemmed)
			return output


In [12]:
from modulenorm.modNormalize import normalize
from modulenorm.modTokenizing import tokenize

data_train = pd.read_csv('./test.txt')
#data_train.columns = ["tanggal", "tweet"]

# menampilkan
no = 1
for row in data_train:
	text = row[2].encode("utf-8")
	text_decode = str(text.decode("utf-8"))

	usenorm = normalize()
	text_norm = usenorm.enterNormalize(text_decode) # normalisasi enter, 1 revw 1 baris
	# text_norm = usenorm.lowerNormalize(text_norm) # normalisasi huruf besar ke kecil
	text_norm = usenorm.repeatcharNormalize(text_norm) # normalisasi titik yang berulang
	text_norm = usenorm.linkNormalize(text_norm) # normalisasi link dalam text
	text_norm = usenorm.spacecharNormalize(text_norm) # normalisasi spasi karakter
	text_norm = usenorm.ellipsisNormalize(text_norm) # normalisasi elepsis (…)

	tok = tokenize() # panggil modul tokenisasi
	text_norm = tok.WordTokenize(text_norm) # pisah tiap kata pada kalimat

	text_norm = usenorm.spellNormalize(text_norm) # cek spell dari kata perkata
	text_norm = usenorm.wordcNormalize(text_norm,2) # menyambung kata (malam-malam) (param: textlist, jmlh_loop)
	# text_norm = usenorm.stemmingNormalize(text_norm,'word') # mengubah ke bentuk kata dasar (text, type_data)

	text_norm = ' '.join(text_norm) # menggabung kalimat tokenize dengan separate spasi

	text_norm = usenorm.emoticonNormalize(text_norm) # menggabung struktur emoticon yang terpisah ([: - )] = [:-)])

	# walking2
	# konfer @ ke at untuk penunjuk tempat

	output = open("output.txt","a")
	output.write(str(text_norm))
	output.write('\n')
	output.close()
	
	# print(no)
	no += 1

# tutup koneksi
#cur.close()
#conn.close()

In [13]:
data_train.head()

,im feeling rather rotten so im not very ambitious right now;sadness
0,im updating my blog because i feel shitty;sadness
1,i never make her separate from me because i do...
2,i left with my bouquet of red and yellow tulip...
3,i was feeling a little vain when i did this on...
4,i cant walk into a shop anywhere where i do no...


In [22]:
data_train.to_csv("normal.csv", index=False)